# 국민은행

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 드라이버 경로 설정
driver = webdriver.Chrome()

# 웹페이지로 이동
driver.get('https://omoney.kbstar.com/quics?page=C016505')

# 페이지 로드 대기
time.sleep(3)

# 1. 검색 창에 '서울' 입력
search_box = driver.find_element(By.XPATH, r'//*[@id="cong.searchName"]')  # 검색창 ID는 실제 확인이 필요함
search_box.send_keys('서울특별시')  # '서울' 입력
search_box.send_keys(Keys.RETURN)  # Enter 키로 검색 실행

# 페이지 로드 대기
time.sleep(3)

# 데이터를 저장할 리스트 생성
all_data = []

next_button = driver.find_element(By.XPATH, f'//*[@id="cong_paging"]/span[6]/button')
next_button.click()

# 2. 페이지 수집 루프
page = 4
while True:
    # 페이지가 로드될 시간을 확보
    time.sleep(3)
    
    # 3. 현재 페이지의 표 데이터를 가져오기
    try:
        table = driver.find_element(By.XPATH, f'//*[@id="search_list"]')
        html_content = table.get_attribute('outerHTML')
        soup = BeautifulSoup(html_content, 'html.parser')
        
        for li in soup.find_all('li'):
            branch_name = li.find('strong', class_='tit').find('a').text.strip()
    
            # 도로명 주소 추출
            road_address = li.find('dl', class_='info2').find_all('dd')[0].text.strip()
            
            # 데이터 리스트에 추가
            all_data.append([branch_name, road_address])
        
        print(f"Page {page} data collected")
    except:
        print(f"Error occurred on page {page}")
        break

    # 4. 다음 페이지로 이동
    try:
        if page % 5 == 3 and page > 3:  # 5의 배수일 때는 '다음' 버튼을 클릭
            next_button = driver.find_element(By.XPATH, f'//*[@id="cong_paging"]/span[{page}]/button')
            next_button.click()
            print(f"'다음' 버튼 클릭 (page {page})")
            page=2
        else:
            # 페이지 번호를 클릭하여 이동
            page_button = driver.find_element(By.XPATH, f'//*[@id="cong_paging"]/span[{page}]/input')
            page_button.click()
            print(f"{page}페이지로 이동 완료")
        
        page += 1
        time.sleep(3)  # 페이지 로딩 대기 (필요시 조정)
    except:
        # 더 이상 '다음' 버튼이 없으면 종료
        print("No more pages.")
        break

# 5. pandas 데이터프레임으로 변환
df = pd.DataFrame(all_data, columns=['영업점명', '도로명주소'])

# 데이터프레임 출력
print(df)

# 크롬 드라이버 종료
driver.quit()

Page 4 data collected
4페이지로 이동 완료
Page 5 data collected
5페이지로 이동 완료
Page 6 data collected
6페이지로 이동 완료
Page 7 data collected
7페이지로 이동 완료
Page 8 data collected
'다음' 버튼 클릭 (page 8)
Page 3 data collected
3페이지로 이동 완료
Page 4 data collected
4페이지로 이동 완료
Page 5 data collected
5페이지로 이동 완료
Page 6 data collected
6페이지로 이동 완료
Page 7 data collected
7페이지로 이동 완료
Page 8 data collected
'다음' 버튼 클릭 (page 8)
Page 3 data collected
3페이지로 이동 완료
Page 4 data collected
4페이지로 이동 완료
Page 5 data collected
5페이지로 이동 완료
Page 6 data collected
6페이지로 이동 완료
Page 7 data collected
7페이지로 이동 완료
Page 8 data collected
'다음' 버튼 클릭 (page 8)
Page 3 data collected
3페이지로 이동 완료
Page 4 data collected
4페이지로 이동 완료
Page 5 data collected
5페이지로 이동 완료
Page 6 data collected
6페이지로 이동 완료
Page 7 data collected
7페이지로 이동 완료
Page 8 data collected
'다음' 버튼 클릭 (page 8)
Page 3 data collected
3페이지로 이동 완료
Page 4 data collected
4페이지로 이동 완료
Page 5 data collected
5페이지로 이동 완료
Page 6 data collected
6페이지로 이동 완료
Page 7 data collected
7페이지로 이동 완료
Page 8 data coll

In [46]:
# 첫 페이지일 때마다 중복해서 수집된 것을 확인 55개개
df.drop_duplicates('영업점명', keep='first', inplace=True)

In [52]:
df_remain = {'영업점명':['명동영업부','남대문종합금융센터','중부','동대문패션타운','명동역'],
 '도로명주소':['서울 중구 을지로 51 (을지로2가)','서울특별시 중구 세종대로 50,(남대문로4가)','서울특별시 중구 충무로 29,(초동)','서울특별시 중구 장충단로13길 34,2층(을지로6가,동화상가)','서울특별시 중구 퇴계로 131,(충무로2가,신일빌딩)']}

In [54]:
df_remain = pd.DataFrame(df_remain)

In [56]:
df_final = pd.concat([df_remain, df], axis=0)

In [64]:
df_final.reset_index(inplace=True)
df_final.drop('index', axis=1, inplace=True)

In [57]:
import requests
import pandas as pd

# 네이버 API 클라이언트 정보
client_id = "3fmd7xia0h"
client_secret = "uy5gOT87pjBgIeAmXCH2dpXVBKK090HceaprqeKL"

# API 요청 함수 정의
def get_coordinates(address):
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    url = f"{endpoint}?query={address}"

    # API 요청 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # API 호출 및 좌표 추출
    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        data = res.json()
        if data['status'] == 'OK' and data['meta']['totalCount'] > 0:
            x = data['addresses'][0]['x']
            y = data['addresses'][0]['y']
            return pd.Series([x, y])
    return pd.Series([None, None])

In [58]:
# 주소 열에 대해 좌표를 적용하여 x, y 좌표 열 추가
df_final[['x', 'y']] = df_final['도로명주소'].apply(get_coordinates)

# 결과 확인
df_final.head()

,영업점명,도로명주소,x,y
0,명동영업부,서울 중구 을지로 51 (을지로2가),126.9836985,37.5664963
1,남대문종합금융센터,"서울특별시 중구 세종대로 50,(남대문로4가)",126.9762504,37.5609215
2,중부,"서울특별시 중구 충무로 29,(초동)",126.9925083,37.5638367
3,동대문패션타운,"서울특별시 중구 장충단로13길 34,2층(을지로6가,동화상가)",127.0069194,37.5686488
4,명동역,"서울특별시 중구 퇴계로 131,(충무로2가,신일빌딩)",126.9864204,37.5612798


In [65]:
df_final[df_final['x'].isna()]

,영업점명,도로명주소,x,y
271,KT광화문(점),,None,None


In [67]:
df_final.iloc[271, 2] = 126.9788661
df_final.iloc[271, 3] = 37.5720274

In [68]:
df_final[df_final['x'].isna()]

,영업점명,도로명주소,x,y


In [72]:
df_final.to_csv('kookmin_bank.csv')

In [14]:
import pandas as pd
kookmin = pd.read_csv('kookmin_bank.csv', index_col='Unnamed: 0')

In [6]:
len(kookmin)

291

In [16]:
kookmin_name = []
for i in range(len(kookmin)):
    kookmin_name.append('국민은행')

In [18]:
kookmin_name = pd.DataFrame(kookmin_name)

In [13]:
kookmin

,은행
0,국민은행
1,국민은행
2,국민은행
3,국민은행
4,국민은행
...,...
286,국민은행
287,국민은행
288,국민은행
289,국민은행


In [20]:
kookmin_bank = pd.concat([kookmin_name, kookmin], axis=1)

In [21]:
kookmin_bank.columns = ['은행','지점','주소','x','y']

In [22]:
kookmin_bank

,은행,지점,주소,x,y
0,국민은행,명동영업부,서울 중구 을지로 51 (을지로2가),126.983699,37.566496
1,국민은행,남대문종합금융센터,"서울특별시 중구 세종대로 50,(남대문로4가)",126.976250,37.560921
2,국민은행,중부,"서울특별시 중구 충무로 29,(초동)",126.992508,37.563837
3,국민은행,동대문패션타운,"서울특별시 중구 장충단로13길 34,2층(을지로6가,동화상가)",127.006919,37.568649
4,국민은행,명동역,"서울특별시 중구 퇴계로 131,(충무로2가,신일빌딩)",126.986420,37.561280
...,...,...,...,...,...
286,국민은행,목동PB센터,"서울특별시 양천구 목동동로 381,2층(목동)",126.879045,37.535736
287,국민은행,잠실롯데PB센터,"서울특별시 송파구 올림픽로 240,4층(잠실동,롯데월드)",127.098164,37.511079
288,국민은행,스타시티PB센터,"서울특별시 광진구 능동로 90,A동2층(자양동)",127.070972,37.538630
289,국민은행,강남스타PB센터,"서울특별시 강남구 테헤란로 152,21층(역삼동,강남파이낸스센터)",127.036546,37.500026


In [23]:
kookmin_bank.to_csv('kookmin_bank.csv')